In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['HC', 'HCs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('healthy controls', 315),
 ('hydrocortisone', 290),
 ('heavy chain', 270),
 ('hippocampus', 116),
 ('hemorrhagic cystitis', 79),
 ('hair cells', 73),
 ('high cholesterol', 70),
 ('high carbohydrate', 56),
 ('hypercholesterolemia', 55),
 ('hip circumference', 50),
 ('high concentrate', 45),
 ('head circumference', 43),
 ('haptocorrin', 43),
 ('hormonal contraception', 41),
 ('hepatocytes', 34),
 ('heavy chains', 33),
 ('homocysteine', 29),
 ('hydrogen cyanamide', 26),
 ('sensory hair cells', 25),
 ('hydroxycarbamide', 24),
 ('high cholesterol diet', 24),
 ('hypercholesterolemic', 23),
 ('hydrocarbon', 23),
 ('heterochromatin', 18),
 ('haemorrhagic cystitis', 15),
 ('hemichannel', 15),
 ('horizontal cells', 14),
 ('hydroxychavicol', 14),
 ('hemorrhagic colitis', 13),
 ('hydrodynamic cavitation', 13),
 ('human chondrocytes', 13),
 ('hormonal contraceptives', 13),
 ('high calorie', 12),
 ('human complement', 11),
 ('hypercholesterolaemic', 11),
 ('heparin cofactor ii', 11),
 ('healthy co

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'CHEBI:CHEBI:17650': 'cortisol',
 'MESH:D006624': 'Hippocampus',
 'DOID:DOID:0050859': 'hemorrhagic cystitis',
 'CHEBI:CHEBI:16646': 'carbohydrate',
 'MESH:D006937': 'Hypercholesterolemia',
 'HGNC:11652': 'TCN1',
 'MESH:D000080282': 'Hormonal Contraception',
 'MESH:D022781': 'Hepatocytes',
 'CHEBI:CHEBI:17230': 'homocysteine',
 'CHEBI:CHEBI:16698': 'cyanamide',
 'CHEBI:CHEBI:44423': 'hydroxyurea',
 'MESH:D004032': 'Diet',
 'CHEBI:CHEBI:24632': 'hydrocarbon',
 'GO:GO:0000792': 'heterochromatin',
 'MESH:D003556': 'Cystitis',
 'MESH:C051268': '2-hydroxychavicol',
 'MESH:D003092': 'Colitis',
 'MESH:D003165': 'Complement System Proteins',
 'HGNC:4838': 'SERPIND1',
 'MESH:D002821': 'Chorioamnionitis',
 'MESH:D002312': 'Cardiomyopathy, Hypertrophic',
 'MESH:D006935': 'Hypercapnia',
 'MESH:D002645': 'Chickens',
 'MESH:D019902': 'Chondrocytes',
 'MESH:D018285': 'Klatskin Tumor',
 'DOID:DOID:684': 'hepatocellular carcinoma',
 'CHEBI:CHEBI:5801': 'hydroxychloroquine',
 'MESH:C070019': 'hypodermi

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 04:46:53] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'cholesterol enriched diet': 'ungrounded',
  'ha tagged wt cah1': 'ungrounded',
  'haemochromatosis': 'MESH:D006432',
  'haemorrhagic cystitis': 'ungrounded',
  'hair cells': 'MESH:D006198',
  'hair color': 'MESH:D006200',
  'hair cycle': 'GO:GO:0042633',
  'hairy cells': 'ungrounded',
  'half crossover': 'ungrounded',
  'half crossovers': 'ungrounded',
  'haptocorrin': 'HGNC:11652',
  'hard conjugates': 'ungrounded',
  'hatchery treated chickens': 'ungrounded',
  'hc': 'ungrounded',
  'hc030031': 'ungrounded',
  'hc067047': 'MESH:C573308',
  'head circumference': 'head_circumference',
  'health centres': 'ungrounded',
  'health clusters': 'ungrounded',
  'health coaching': 'ungrounded',
  'health controls': 'MESH:D064368',
  'healthy comparison': 'ungrounded',
  'healthy controls': 'MESH:D064368',
  'healthy controls subjects': 'MESH:D064368',
  'healthy subjects': 'MESH:D064368',
  'heat capacity': 'ungrounded',
  'heat not burn cigarette': 'ungrounded',
  'heavy chain': 'heavy_cha

In [11]:
grounding_map, names, pos_labels = [{'cholesterol enriched diet': 'ungrounded',
  'ha tagged wt cah1': 'ungrounded',
  'haemochromatosis': 'MESH:D006432',
  'haemorrhagic cystitis': 'ungrounded',
  'hair cells': 'MESH:D006198',
  'hair color': 'MESH:D006200',
  'hair cycle': 'GO:GO:0042633',
  'hairy cells': 'ungrounded',
  'half crossover': 'ungrounded',
  'half crossovers': 'ungrounded',
  'haptocorrin': 'HGNC:11652',
  'hard conjugates': 'ungrounded',
  'hatchery treated chickens': 'ungrounded',
  'hc': 'ungrounded',
  'hc030031': 'ungrounded',
  'hc067047': 'MESH:C573308',
  'head circumference': 'head_circumference',
  'health centres': 'ungrounded',
  'health clusters': 'ungrounded',
  'health coaching': 'ungrounded',
  'health controls': 'MESH:D064368',
  'healthy comparison': 'ungrounded',
  'healthy controls': 'MESH:D064368',
  'healthy controls subjects': 'MESH:D064368',
  'healthy subjects': 'MESH:D064368',
  'heat capacity': 'ungrounded',
  'heat not burn cigarette': 'ungrounded',
  'heavy chain': 'heavy_chain',
  'heavy chains': 'heavy_chain',
  'helminthosporium carbonum': 'ungrounded',
  'helper component': 'IP:IPR001456',
  'hematopoietic cells': 'MESH:D006412',
  'hemicastrated': 'ungrounded',
  'hemichannel': 'ungrounded',
  'hemicolectomy': 'ungrounded',
  'hemochromatosis': 'MESH:D006432',
  'hemocoagulase': 'ungrounded',
  'hemorrhagic colitis': 'ungrounded',
  'hemorrhagic cystitis': 'ungrounded',
  'heparin cofactor ii': 'HGNC:4838',
  'hepatocellular carcinoma': 'DOID:DOID:684',
  'hepatocytes': 'MESH:D022781',
  'heterochromatin': 'GO:GO:0000792',
  'heteroconjugate': 'ungrounded',
  'hierarchical clustering': 'MESH:D016000',
  'high calorie': 'ungrounded',
  'high calorie diet': 'ungrounded',
  'high carbohydrate': 'high_carbohydrate',
  'high carbohydrate overfeeding': 'high_carbohydrate',
  'high cholesterol': 'MESH:D006937',
  'high cholesterol diet': 'MESH:D006937',
  'high concentrate': 'high_concentrate',
  'high concentrate diet': 'high_concentrate',
  'high confidence': 'ungrounded',
  'high contacts': 'ungrounded',
  'hilar cholangiocarcinoma': 'MESH:D018285',
  'hip circumference': 'hip_circumference',
  'hippocampal': 'MESH:D006624',
  'hippocampus': 'MESH:D006624',
  'histologic chorioamnionitis': 'MESH:D002821',
  'hofbauer cells': 'ungrounded',
  'holding currents': 'ungrounded',
  'home cage controls': 'ungrounded',
  'homocysteine': 'CHEBI:CHEBI:17230',
  'hook complex': 'IP:IPR001624',
  'horizontal cells': 'MESH:D051248',
  'hormonal contraception': 'MESH:D000080282',
  'hormonal contraceptives': 'MESH:D000080282',
  'household contacts': 'ungrounded',
  'houttuynia cordata': 'ungrounded',
  'houttuynia cordata thunb': 'ungrounded',
  'htlv 1 carriers': 'ungrounded',
  'huang c': 'ungrounded',
  'human capital': 'ungrounded',
  'human chondrocytes': 'MESH:D019902',
  'human complement': 'MESH:D003165',
  'huntington s chorea': 'MESH:D006816',
  'hyaluronan chitosan': 'ungrounded',
  'hycanthone': 'CHEBI:CHEBI:52768',
  'hydro cortisone': 'CHEBI:CHEBI:17650',
  'hydrocarbon': 'CHEBI:CHEBI:24632',
  'hydrocodone': 'CHEBI:CHEBI:5779',
  'hydrocortisone': 'CHEBI:CHEBI:17650',
  'hydrocortisone acetate': 'MESH:C021650',
  'hydrocortisone sodium succinate': 'CHEBI:CHEBI:5782',
  'hydrodynamic cavitation': 'ungrounded',
  'hydrogen concentrate': 'ungrounded',
  'hydrogen cyanamide': 'CHEBI:CHEBI:16698',
  'hydrolysed casein': 'ungrounded',
  'hydrolyzed casein': 'ungrounded',
  'hydrolyzed collagen': 'ungrounded',
  'hydrophobic core': 'ungrounded',
  'hydroxy chavicol': 'CHEBI:CHEBI:50158',
  'hydroxycarbamide': 'CHEBI:CHEBI:44423',
  'hydroxychavicol': 'CHEBI:CHEBI:50158',
  'hydroxychloroquine': 'CHEBI:CHEBI:5801',
  'hydroxycholesterol': 'MESH:D006888',
  'hydroxycinnamic': 'ungrounded',
  'hydroxycitrate': 'MESH:C007999',
  'hypercalcemia': 'MESH:D006934',
  'hypercapnia': 'MESH:D006935',
  'hypercholesterolaemia': 'MESH:D006937',
  'hypercholesterolaemic': 'MESH:D006937',
  'hypercholesterolemia': 'MESH:D006937',
  'hypercholesterolemic': 'MESH:D006937',
  'hypercholesterolemic diet': 'MESH:D006937',
  'hypercortisolism': 'MESH:D000308',
  'hypertrophic cardiomyopathy': 'MESH:D002312',
  'hyphal compartment': 'ungrounded',
  'hypodermin c': 'MESH:C070019',
  'hypothyroid coma': 'ungrounded',
  'hypoxic conditions': 'ungrounded',
  'sensory hair cells': 'MESH:D006198'},
 {'MESH:D006432': 'Hemochromatosis',
  'MESH:D006198': 'Hair Cells, Auditory',
  'MESH:D006200': 'Hair Color',
  'GO:GO:0042633': 'hair cycle',
  'HGNC:11652': 'TCN1',
  'MESH:C573308': 'HC-067047',
  'head_circumference': 'head_circumference',
  'MESH:D064368': 'Healthy Volunteers',
  'heavy_chain': 'heavy_chain',
  'IP:IPR001456': 'Helper component proteinase',
  'MESH:D006412': 'Hematopoietic Stem Cells',
  'DOID:DOID:0050859': 'hemorrhagic cystitis',
  'HGNC:4838': 'SERPIND1',
  'DOID:DOID:684': 'hepatocellular carcinoma',
  'MESH:D022781': 'Hepatocytes',
  'GO:GO:0000792': 'heterochromatin',
  'MESH:D016000': 'Cluster Analysis',
  'high_carbohydrate': 'high_carbohydrate',
  'MESH:D006937': 'Hypercholesterolemia',
  'high_concentrate': 'high_concentrate',
  'MESH:D018285': 'Klatskin Tumor',
  'hip_circumference': 'hip_circumference',
  'MESH:D006624': 'Hippocampus',
  'MESH:D002821': 'Chorioamnionitis',
  'CHEBI:CHEBI:17230': 'homocysteine',
  'IP:IPR001624': 'Flagellar hook-basal body complex protein FliE',
  'MESH:D051248': 'Retinal Horizontal Cells',
  'MESH:D000080282': 'Hormonal Contraception',
  'MESH:D019902': 'Chondrocytes',
  'MESH:D003165': 'Complement System Proteins',
  'MESH:D006816': 'Huntington Disease',
  'CHEBI:CHEBI:52768': 'hycanthone',
  'CHEBI:CHEBI:17650': 'cortisol',
  'CHEBI:CHEBI:24632': 'hydrocarbon',
  'CHEBI:CHEBI:5779': 'hydrocodone',
  'MESH:C021650': 'hydrocortisone acetate',
  'CHEBI:CHEBI:5782': 'Hydrocortisone Sodium Succinate',
  'CHEBI:CHEBI:16698': 'cyanamide',
  'CHEBI:CHEBI:50158': 'chavicol',
  'CHEBI:CHEBI:44423': 'hydroxyurea',
  'MESH:C051268': '2-hydroxychavicol',
  'CHEBI:CHEBI:5801': 'hydroxychloroquine',
  'MESH:D006888': 'Hydroxycholesterols',
  'MESH:C007999': 'hydroxycitric acid',
  'MESH:D006934': 'Hypercalcemia',
  'MESH:D006935': 'Hypercapnia',
  'MESH:D000308': 'Adrenocortical Hyperfunction',
  'MESH:D002312': 'Cardiomyopathy, Hypertrophic',
  'MESH:C070019': 'hypodermin C'},
 ['CHEBI:CHEBI:16698',
  'CHEBI:CHEBI:17230',
  'CHEBI:CHEBI:17650',
  'CHEBI:CHEBI:44423',
  'DOID:DOID:0050859',
  'HGNC:11652',
  'HGNC:4838',
  'MESH:D006198',
  'MESH:D006624',
  'MESH:D006937',
  'MESH:D022781']]

In [12]:
excluded_longforms = ['hc']

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [19]:
intersection1

[]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [20]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 04:53:03] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 04:54:34] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.8257419878512395 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [21]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:17650': 229,
  'ungrounded': 230,
  'MESH:D006432': 4,
  'heavy_chain': 280,
  'CHEBI:CHEBI:5782': 5,
  'MESH:D006198': 103,
  'MESH:D002312': 10,
  'MESH:D006937': 133,
  'HGNC:4838': 10,
  'MESH:D006934': 4,
  'MESH:D022781': 26,
  'CHEBI:CHEBI:24632': 22,
  'MESH:C021650': 4,
  'CHEBI:CHEBI:50158': 12,
  'MESH:D006816': 5,
  'CHEBI:CHEBI:52768': 1,
  'HGNC:11652': 28,
  'MESH:D064368': 196,
  'MESH:C070019': 3,
  'MESH:C007999': 3,
  'IP:IPR001456': 6,
  'head_circumference': 33,
  'CHEBI:CHEBI:17230': 25,
  'high_carbohydrate': 30,
  'MESH:D051248': 17,
  'CHEBI:CHEBI:5779': 3,
  'DOID:DOID:684': 4,
  'CHEBI:CHEBI:16698': 12,
  'MESH:D006624': 77,
  'CHEBI:CHEBI:44423': 16,
  'MESH:D000080282': 38,
  'high_concentrate': 23,
  'hip_circumference': 32,
  'GO:GO:0000792': 16,
  'MESH:D002821': 6,
  'MESH:D006935': 8,
  'MESH:D006412': 5,
  'GO:GO:0042633': 3,
  'MESH:D019902': 8,
  'MESH:C573308': 3,
  'MESH:D018285': 3,
  'MESH:D006200': 1,
  'MES

In [22]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [23]:
disamb.dump(model_name, results_path)

In [24]:
print(disamb.info())

Disambiguation model for HC, and HCs

Produces the disambiguations:
	2-hydroxychavicol	MESH:C051268
	Adrenocortical Hyperfunction	MESH:D000308
	Cardiomyopathy, Hypertrophic	MESH:D002312
	Chondrocytes	MESH:D019902
	Chorioamnionitis	MESH:D002821
	Cluster Analysis	MESH:D016000
	Complement System Proteins	MESH:D003165
	Flagellar hook-basal body complex protein FliE	IP:IPR001624
	HC-067047	MESH:C573308
	Hair Cells, Auditory*	MESH:D006198
	Hair Color	MESH:D006200
	Healthy Volunteers	MESH:D064368
	Helper component proteinase	IP:IPR001456
	Hematopoietic Stem Cells	MESH:D006412
	Hemochromatosis	MESH:D006432
	Hepatocytes*	MESH:D022781
	Hippocampus*	MESH:D006624
	Hormonal Contraception	MESH:D000080282
	Huntington Disease	MESH:D006816
	Hydrocortisone Sodium Succinate	CHEBI:CHEBI:5782
	Hydroxycholesterols	MESH:D006888
	Hypercalcemia	MESH:D006934
	Hypercapnia	MESH:D006935
	Hypercholesterolemia*	MESH:D006937
	Klatskin Tumor	MESH:D018285
	Retinal Horizontal Cells	MESH:D051248
	SERPIND1*	HGNC:4838
	TCN

In [25]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1